In [1]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Embedding, Conv1D, MaxPooling1D, LSTM
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pickle
import numpy as np

In [2]:
# Load the reviews and labels from the file
with open('train_reviews.pkl', 'rb') as f:
    train_reviews, train_labels = pickle.load(f)

with open('test_reviews.pkl', 'rb') as f:
    test_reviews, test_labels = pickle.load(f)

In [3]:
# Load the tokenized and padded sequences
x_train = np.load('x_train.npy')
x_test = np.load('x_test.npy')

# Load the one-hot encoded labels
y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')

# Load the tokenizer
with open('tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

max_words = 10000
max_review_length = 500

In [ ]:
# Define the LSTM-CNN model architecture
embedding_vector_length = 32
model = Sequential()
model.add(Embedding(max_words, embedding_vector_length, input_length=max_review_length))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(10, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=5, batch_size=64)

# Save the model
model.save('LSTM_CNN_model.keras')

: 

In [ ]:
# Load the model
model = load_model('LSTM_CNN_model.keras')

: 

In [6]:
# Predict the test set
y_pred = model.predict(x_test)

# Convert predictions to class labels
y_pred_labels = [1 if pred[1] > pred[0] else 0 for pred in y_pred]

# Calculate and print accuracy, precision, recall, and F1 score
accuracy = accuracy_score(test_labels, y_pred_labels)
precision = precision_score(test_labels, y_pred_labels)
recall = recall_score(test_labels, y_pred_labels)
f1 = f1_score(test_labels, y_pred_labels)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

782/782 ━━━━━━━━━━━━━━━━━━━━ 53s 67ms/step
Accuracy: 0.87
Precision: 0.91
Recall: 0.82
F1 Score: 0.86


In [11]:
# Take user input for the review
input_review = input("Enter a review: ")

# Preprocessing function
def preprocess_review(review, tokenizer, maxlen):
    # Convert the review to a sequence of integers
    sequences = tokenizer.texts_to_sequences([review])
    # Pad the sequences to ensure uniform input length
    padded_sequences = pad_sequences(sequences, maxlen=maxlen)
    return padded_sequences

# Preprocess the input review
preprocessed_review = preprocess_review(input_review, tokenizer, max_review_length)

# Predict the sentiment of the input review
y_pred = model.predict(preprocessed_review)

# Convert prediction to class label (1 for positive, 0 for negative)
y_pred_label = 1 if y_pred[0][1] > y_pred[0][0] else 0

# Print the predicted sentiment
sentiment = "positive" if y_pred_label == 1 else "negative"
print(f"The predicted sentiment for the review is: {sentiment}")

Enter a review: I was pleasantly surprised to find this movie showing as a sneak preview in my local theater.<br /><br />We have all seen this plot line before (Top Gun, GI Jane, An Officer and a Gentleman) but a good script still works. This story is basically about the training of a Coast Guard rescue team with a couple of side story lines. Kevin Costner plays a highly successful rescue team leader, Ben Randall, who is forced into heading the training team after a tough mission. The movie takes us through the rigors of the training process and the personal stories of both the Costner character and that of Jake Fischer, played by Ashton Kutcher. I am happy to say that Ashton is great in this part.<br /><br />There are no great surprises in this movie and you will probably realize what is coming long before it arrives. However, the use of humor, the exploration of the toughness of the training and the fun of watching Ben Randall "do his own thing as a trainer", kept me riveted and thor